In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

In [2]:
# Read the data
df = pd.read_csv('WA_Fn-UseC_-Telco-Customer-Churn.csv')
target = 'churn'
df.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [3]:
# let's drop the column customerID
del df['customerID']

#### Peparing and cleaning data

In [4]:
df.head().T

,0,1,2,3,4
gender,Female,Male,Male,Male,Female
SeniorCitizen,0,0,0,0,0
Partner,Yes,No,No,No,No
Dependents,No,No,No,No,No
tenure,1,34,2,45,2
PhoneService,No,Yes,Yes,No,Yes
MultipleLines,No phone service,No,No,No phone service,No
InternetService,DSL,DSL,DSL,DSL,Fiber optic
OnlineSecurity,No,Yes,Yes,Yes,No
OnlineBackup,Yes,No,Yes,No,No


In [6]:
# standarize data, replace spaces by _ in Column anmes and data

df.columns = df.columns.str.lower().str.replace(' ','_')

rows = []
for col in df.columns:
    rows.append([col,df[col].dtype, df[col].unique()]) 
pd.DataFrame(rows, columns=['Feature', 'Type', 'Unique Values'])

,Feature,Type,Unique Values
0,gender,object,"[Female, Male]"
1,seniorcitizen,int64,"[0, 1]"
2,partner,object,"[Yes, No]"
3,dependents,object,"[No, Yes]"
4,tenure,int64,"[1, 34, 2, 45, 8, 22, 10, 28, 62, 13, 16, 58, ..."
5,phoneservice,object,"[No, Yes]"
6,multiplelines,object,"[No phone service, No, Yes]"
7,internetservice,object,"[DSL, Fiber optic, No]"
8,onlinesecurity,object,"[No, Yes, No internet service]"
9,onlinebackup,object,"[Yes, No, No internet service]"


In [7]:
# Checking Column totalcharges
print('Intended to see error: \"ValueError: Unable to parse string \"_\" at position 488\"')
pd.to_numeric(df.totalcharges)

Intended to see error: "ValueError: Unable to parse string "_" at position 488"


ValueError: Unable to parse string " " at position 488

In [8]:
# Error shows a '_' that can't be converted
df.totalcharges = pd.to_numeric(df.totalcharges, errors='coerce')

In [9]:
# Characters that couldn't be converted to numerical values show as nan
print(f'Null values for column totalcharges: {df.totalcharges.isnull().sum()}')

# Filling missing values with zero
df.totalcharges.fillna(0, inplace=True)
print(f'Null values for column totalcharges after converting to numerical and fillin NAs: {df.totalcharges.isnull().sum()}')

Null values for column totalcharges: 11
Null values for column totalcharges after converting to numerical and fillin NAs: 0


In [10]:
categCols = df.select_dtypes('object').columns
for col in categCols:
    df[col] = df[col].str.lower().str.replace(' ','_')
#print(f'Categorical columns: {categCols}\n')
df.head()

,gender,seniorcitizen,partner,dependents,tenure,phoneservice,multiplelines,internetservice,onlinesecurity,onlinebackup,deviceprotection,techsupport,streamingtv,streamingmovies,contract,paperlessbilling,paymentmethod,monthlycharges,totalcharges,churn
0,female,0,yes,no,1,no,no_phone_service,dsl,no,yes,no,no,no,no,month-to-month,yes,electronic_check,29.85,29.85,no
1,male,0,no,no,34,yes,no,dsl,yes,no,yes,no,no,no,one_year,no,mailed_check,56.95,1889.50,no
2,male,0,no,no,2,yes,no,dsl,yes,yes,no,no,no,no,month-to-month,yes,mailed_check,53.85,108.15,yes
3,male,0,no,no,45,no,no_phone_service,dsl,yes,no,yes,yes,no,no,one_year,no,bank_transfer_(automatic),42.30,1840.75,no
4,female,0,no,no,2,yes,no,fiber_optic,no,no,no,no,no,no,month-to-month,yes,electronic_check,70.70,151.65,yes


In [11]:
# Converting the target variable to binary values, no=0 or yes=1.
df.churn = (df.churn == 'yes') * 1
df.churn.head()

0    0
1    0
2    1
3    0
4    1
Name: churn, dtype: int32

#### Validation framework

In [12]:
#Split data
from sklearn.model_selection import train_test_split

In [13]:
dfTrainFull, dfTest = train_test_split(df, test_size=0.2, random_state=1)
len(dfTrainFull), len(dfTest)

(5634, 1409)

In [14]:
dfTrain, dfVal = train_test_split(dfTrainFull, test_size=0.25, random_state=1)
len(dfTrain), len(dfVal), len(dfTest)

(4225, 1409, 1409)

In [15]:
dfTrain.reset_index(drop=True, inplace=True)
dfVal.reset_index(drop=True, inplace=True)
dfTest.reset_index(drop=True, inplace=True)

In [16]:
yTrain = dfTrain[target].values
yVal = dfVal[target].values
yTest = dfTest[target].values

In [17]:
del dfTrain[target]
del dfVal[target]
del dfTest[target]

#### EDA

In [18]:
dfTrainFull.reset_index(drop=True, inplace=True)

In [19]:
dfTrainFull.head(2)

,gender,seniorcitizen,partner,dependents,tenure,phoneservice,multiplelines,internetservice,onlinesecurity,onlinebackup,deviceprotection,techsupport,streamingtv,streamingmovies,contract,paperlessbilling,paymentmethod,monthlycharges,totalcharges,churn
0,male,0,yes,yes,12,yes,no,no,no_internet_service,no_internet_service,no_internet_service,no_internet_service,no_internet_service,no_internet_service,two_year,no,mailed_check,19.7,258.35,0
1,female,0,no,no,42,yes,no,dsl,yes,yes,yes,yes,no,yes,one_year,no,credit_card_(automatic),73.9,3160.55,1


In [20]:
# Churn rate in %
totalChurn = dfTrainFull[target].value_counts(normalize=True)[1].round(3)
totalChurn

0.27

In [42]:
# Convert feature seniorcitizen to object to treat it as categorical
dfTrainFull.seniorcitizen = dfTrainFull.seniorcitizen.astype('object')

In [22]:
dfTrainFull.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5634 entries, 0 to 5633
Data columns (total 20 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   gender            5634 non-null   object 
 1   seniorcitizen     5634 non-null   object 
 2   partner           5634 non-null   object 
 3   dependents        5634 non-null   object 
 4   tenure            5634 non-null   int64  
 5   phoneservice      5634 non-null   object 
 6   multiplelines     5634 non-null   object 
 7   internetservice   5634 non-null   object 
 8   onlinesecurity    5634 non-null   object 
 9   onlinebackup      5634 non-null   object 
 10  deviceprotection  5634 non-null   object 
 11  techsupport       5634 non-null   object 
 12  streamingtv       5634 non-null   object 
 13  streamingmovies   5634 non-null   object 
 14  contract          5634 non-null   object 
 15  paperlessbilling  5634 non-null   object 
 16  paymentmethod     5634 non-null   object 


In [23]:
#Check unique values foir the categorical variables
dfTrainFull.select_dtypes('object').nunique()

gender              2
seniorcitizen       2
partner             2
dependents          2
phoneservice        2
multiplelines       3
internetservice     3
onlinesecurity      3
onlinebackup        3
deviceprotection    3
techsupport         3
streamingtv         3
streamingmovies     3
contract            3
paperlessbilling    2
paymentmethod       4
dtype: int64

#### Intuition on feature importance, churn rate and risk ratio

In [24]:
def ChurnCalculations(dfT, col,totalChurn='1'):
    
    dfRisk = dfT.groupby(col).churn.agg(['mean', 'count'])
    dfRisk['diff'] = dfRisk['mean'] - totalChurn
    dfRisk['risk'] = dfRisk['mean'] / totalChurn
    
    return dfRisk

In [25]:
from IPython.display import display

In [26]:
print()
print(f'Total churn rate: {totalChurn}\n')

for col in categCols[:-1]:
    print(f'{col}')
    churn = ChurnCalculations(dfTrainFull, col, totalChurn)
    display(churn)


Total churn rate: 0.27

gender


,mean,count,diff,risk
gender,,,,
female,0.276824,2796,0.006824,1.025274
male,0.263214,2838,-0.006786,0.974865


partner


,mean,count,diff,risk
partner,,,,
no,0.329809,2932,0.059809,1.221515
yes,0.205033,2702,-0.064967,0.759383


dependents


...

,mean,count,diff,risk
contract,,,,
month-to-month,0.431701,3104,0.161701,1.598893
one_year,0.120573,1186,-0.149427,0.446568
two_year,0.028274,1344,-0.241726,0.104718


paperlessbilling


,mean,count,diff,risk
paperlessbilling,,,,
no,0.172071,2313,-0.097929,0.637300
yes,0.338151,3321,0.068151,1.252412


paymentmethod


,mean,count,diff,risk
paymentmethod,,,,
bank_transfer_(automatic),0.168171,1219,-0.101829,0.622854
credit_card_(automatic),0.164339,1217,-0.105661,0.608661
electronic_check,0.455890,1893,0.185890,1.688482
mailed_check,0.193870,1305,-0.076130,0.718036


#### Feature importance: mutual information

In [27]:
from sklearn.metrics import mutual_info_score

In [28]:
mutual_info_score(dfTrainFull.churn, dfTrainFull.contract)

0.0983203874041556

In [33]:
def MutualInfoChurn(data):
    return mutual_info_score(data, dfTrainFull.churn)

In [40]:
mutualInfo = dfTrainFull[categCols[:-1]].apply(MutualInfoChurn)
mutualInfo.sort_values(ascending=False, inplace=True)
mutualInfo

contract            0.098320
onlinesecurity      0.063085
techsupport         0.061032
internetservice     0.055868
onlinebackup        0.046923
deviceprotection    0.043453
paymentmethod       0.043210
streamingtv         0.031853
streamingmovies     0.031581
paperlessbilling    0.017589
dependents          0.012346
partner             0.009968
multiplelines       0.000857
phoneservice        0.000229
gender              0.000117
dtype: float64